<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Overview

In this weeks Lab you will Resample a signal from 44.1 kHz to 16kHz. You will report on the computation time, perception, and SNR.

## Set Up Your Colab Enviornment


First, mount google drive so that you can access the shared class drive and files. (You may want to check the notebooks from lecture for a reminder of how this is done.)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import the libraries you'll be using:


*   `soundfile as sf` for audio signal processing
*   `IPython.display as ipd ` for audio playback
*   `matplotlib as rc` for figure customization
*   `matplotlib.pyplot as plt` for plotting
*   `numpy as np` for some math functions
* `scipy` for STFT calculations
* from `scipy` import `signal` for STFT calculations
* `time` for getting compute time of cells and functions

In [ ]:
#Your code here
import soundfile as sf
import IPython.display as ipd
import matplotlib as rc
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal
import time

Tip: To quickly view documentation for a function, you can use the help function. See below.

In [ ]:
#help(ipd.Audio)

Tip: To quickly and neatly hide code cell outputs, press Ctrl + C + M + O (for Windows) or Cmnd + C + M + O (for Mac).

Run your Helper Functions to be used later in the lab:

### Helper Functions

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt=''):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs
  plt.plot(t[:N],sig[:N],fmt,linewidth=lw)
  plt.xlabel('Time (sec)')
  ax = plt.gca()
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  fig.tight_layout()
  plt.ion()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, x_lim=0, fig_size=(16,4),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)  
  win = np.hanning(n_win)
  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  S_mag = 4*np.abs(S) / n_fft     # Frequency magnitude, normalized by length
                                  #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                  #    x2 because hanning window has 0.5 average
  S_mag += 0.0000001              # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w
  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = fs/2
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.


In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)
  S_mag = 4*np.abs(Sxx) + 0.0000000001    # See myPlotFFT for explanation
  S_dBFS = 20*np.log10(S_mag)
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

### Load the Audio for Lab 4

In [ ]:
load_path = '/content/drive/MyDrive/eces435-work/Labs/Lab4/data/Cello_44_1.wav'
y, fs = sf.read(load_path)
ipd.Audio(y, rate=fs)

# Part 1
Your task is to take the provided 44.1 kHz audio signal and convert it to a sampling rate of 16 kHz. Hopefully you remember from class that to resample a signal by a factor of L/M (where L and M are both integers) you can do the following:

* upsample the signal by a factor of L.
* apply a lowpass (smoothing) filter.
* downsample by a factor of M. <br>

As you work on your resampling system, make sure to listen to the quality of the output signal. Try to make the resampled version sound as good as possible. In addition to preserving audio quality, you should also design with efficiency in mind. Try to minimize the runtime of your code.

For any filter(s) you create, make sure to generate a plot of the filter's magnitude frequency response. In addition to your code and figures, provide a written explanation of your process. Explain how you designed your filter(s) and what steps you took toward the goals of perserving audio quality and minimizing runtime.

You are not permitted to use scipy.signal.resample, scipy.signal.upfirdn, librosa.resample, or any other library functions that perform this resampling process for you.

## a. Upsample the Signal by a Factor of L

Plot the first .002 seconds of the audio using myPlot.

In [ ]:
myPlot()

Upsample the signal y and save the upsampled version to a new variable named `y_up[L]` where L is the integer factor used in upsampling. Hint: Use `np.repeat()` <br>

Plot the first .002 seconds of the new signal to see how it compares to the original. Remember: Now that the signal has been upsampled, the sampling frequency is different.

* Replace '...' with L and M apropriately.

In [ ]:
L = 
y... = 
myPlot()

## b. Apply a Lowpass (Smoothing) Filter

Design a smoothing filter to use on your upsampled signal to mitigate aliasing. First,

* Define an apropriate filter length F_len = 999
* Use np.hanning and create a hanning filter the length of F_len
* Normalize the filter
* Plot the filter using `myPlot` and the magnitude frequency response using `myPlotFFT` with `n_fft = 2*F_len`
* When using `myPlotFFT` set `fs = 2` so that the response will be normalized along the frequency axis (Output plot should have x limits of 0 to 1)

In [ ]:
F_len =                      # What should be the length of our Hann-weighted FIR filter?
h_hann =                     # Use the Hann(ing) function
h_hann =                     # Normalize the filter
myPlot()                     # Plot our filter (Remember sampling freqeuncy is not 44100)
myPlotFFT()                  #Plot Frequency Magnatude response

Use a for loop to impliment convolution of the upsampled signal with the filter you just created. <br>

* Replace '...' with L and M apropriately.
* Same the filtered upsampled signal to a variable `y_up[L]_hann `(Remember: L is the upsampling factor you used)
* This cell will take a while to run. Later in the lab you will need to report how long so go ahead and add look at the runtime section of the lab to see how to time this cells runtime. Belive me, you only want to do this once.

In [ ]:
y_up160_hann = #Initialize the New Filtered Signal as an array of zeros

for n in range():
  y....[n] =  # Convolve upsampled signal with your filter

The method above for implimenting your filter takes a very long time because of the highly inefiecient implimentation. Luckily, there is a much faster way to do this using `signal.lfilter()`. Use `signal.lfilter()` to easily try multiple variations of the filter you designed and figure out the best settings. 
* Replace '...' with L and M apropriately.
* To impliment this fast filter use `signal.lfilter(filter vector,1,signal)`. 
* Be aware, this filter scheme introduces group delay (about 1/2 the window length), so when computing SNR be sure to plot signals and ensure they line up by using padding and cropping.
* Again, look at the Runtime section to see how to time this filtering computation.

In [ ]:
y_up...._lfilter =

## c. Downsample by a Factor of M

Next, you want to complete resampling by downsampling your filtered signal by an integer factor of M. It would also be intersting to compare this signal (upsampled - filtered - downsampled) to a resampled version that does not use filtering and should therefor untroduce a great deal of aliasing. 
* Replace '...' with L and M apropriately.
* First, downsample y_up[L] directly by a factor of M. Save this signal to a variable `y_up[L]_down[M]_aliasing`
* Plot the first .002 seconds of `y_up[L]_down[M]_aliasing` using (Remember fs is the new resampled fs), and listen to it using ipd.Audio. Can you hear the aliasing effects?
* Second, downsample the filtered signal  by a factor of M. Save this signal to a variable `y_up[L]_down[M]`
* Plot the first .002 seconds of `y_up[L]_down[M]` using (Remember fs is the new resampled fs), and listen to it using ipd.Audio.


In [ ]:
y_up..._down..._aliasing = 
myPlot()
ipd.Audio()

In [ ]:
y_up..._down... = 
myPlot()
ipd.Audio()

# Part 2: Evaluate your system

Now, you'll evaluate your system. For a point of comparison, run the following code which loads a 16 kHz resampled version of the signal made using librosa's built-in resample function.

In [ ]:
load_path = '/content/drive/MyDrive/eces435-work/Labs/Lab4/data/Cello_16.wav'
y_16, fs_16 = sf.read(load_path)
ipd.Audio(y_16, rate=fs_16)

## a. Perception

First, you will perform a perceptual evaluation. Listen to your system's 16 kHz output audio and compare it to the 16 kHz version from librosa. Explain any differences you hear.

Now listen to the original 44.1 kHz version again and comment on how the 16 kHz versions differ.

Finally, listen to the 16kHz signal and the resampled signal with aliasing and comment on the differences.

Your response here:

## b. Runtime

You can time how long your resampling code takes to run by sandwiching your code within the following two lines:

In [ ]:
startTime = time.time() # paste this line right before your resampling system

# The code you want to time goes in between

print(time.time() - startTime, "seconds elapsed") # paste this line right after your resampling system

The above code is ideal for loops with multiple lines. When using `signal.lfilter()`, you can add `%time` at the begining of a line to get the runtime of a single line in a code cell, or use the same method as above.

In [ ]:
%time print('Hello') #Gets runtime for a line in the cell

Report your code's runtime below when using the long loop convolution and when using `lfilter`.

Your response here:
* Loop Runtime:
* `signal.lfilter()` Runtime: 

## c. SNR

Signal-to-noise ratio provides a way to measure the similarity between two signals. You will compare your 16 kHz signal to the librosa 16 kHz version.

Follow these steps to compute SNR and report your value in dB below:

* Create a variable "signal" and set it equal to the librosa 16 kHz signal
* Create a variable "noise" and set it equal to the difference (subtraction) between the librosa 16 kHz signal and your 16 kHz signal. (If they are slightly different lengths, pad the shorter one with zeros.)
* Compute the power for the "signal" and "noise". To calculate power, square all the values and sum them.
* Find the signal-to-noise ratio by dividing the signal power over the noise power.
* Finally, compute 10 * np.log10 (ratio) to get the SNR in decibels.
Report your SNR below.


Hint: You can reuse the function from last lab.

Note: SNR is very sensitive to time shifted signals. If your filter(s) introduced a time delay, this could result in a bad SNR. To fix this, you should manually shift the start time of your 16 kHz audio (by padding the start with zeros) so that it lines up with the librosa version as closely as possible. (You will want to zoom in a lot when making sure the signals line up.) SNR is also very sensitive to scaling mismatch so make sure to normalize all input signals to SNR to -1 to 1 id your filter introduced scaling differences.

Check the lengths of the signals:

In [ ]:
print(len())

Plot the signals (very zoomed in) to make sure they line up.

In [ ]:
myPlot()

Crop Signals as nessesarry:

In [ ]:
#Your code here

In [ ]:
# Paste or Create your SNR Function

In [ ]:
# Compute SNR

Your response here:
* SNR between Original and Filtered Signal(Filtered with Loop): 
* SNR between Original and Filtered Signal(Filtered with `signal.lfilter()`): 

# Extra Credit 


In this lab, you resampled a 44.1 KHz signal to 16kHz in one fell swoop. It would be a much better approach to resample the signal in stages by upsampling by factors L of  and down by a factor of M instead of doing it all at once. 

For Extra Credit: 
* Upsample the signal in stages by the factors of L. If L was say 441, then you could upsample with L = 3 twice and then L = 7 twice for a total of upsampling by L = 441. 
* In each stage you need to upsample and then filter to smooth the signal (if you dont smooth in between then upsampling in stages yeilds the same result; here you get to use a smaller filter and create smoother trasnistions between samples)
* You will need to design two filters (one for L = 5 and one for L = 2) and then reuse them as needed.
* After upsampling in multiple stages you should downsample the resulting signal by 441 to yeild the resampled signal. 
* Report on Perception, Runtime, and SNR. 
* Use `signal.lfilter()` here NOT the long way (convolution in a loop)

In [ ]:
# Your code here  (Probably Multiple Cells)

# Completing the Lab

To submit this lab *share the notebook with charis.cochran@excitecenter.org* AND *submit the link as a text submission on the Lab3 assignment on BbLearn* to receive credit for this lab. (ONLY share with charis.cochran@excitecenter.org)

*Ensure all cells and plots have been run and are visible in the notebook before submitting. Also, make sure you responded to the short answer question about the Spectrogram plot. Submiting the link to BbLearn means the lab has been submitted and is ready for grading. DO THIS LAST. 